# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
y[0:5]

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"], axis=1)

# Review the features DataFrame
X.head()

### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X_train_scaled

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train_scaled[0])

# Review the number of features
number_input_features

In [11]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  22

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  11

# Define the number of neurons in the output layer
hidden_nodes_output =  1

In [ ]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='softplus')
)

# Add the second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='softplus')
)

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_output, activation='sigmoid')
)

In [ ]:
# Display the Sequential model summary
nn.summary()

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [17]:
# Set the model's file path
file_path = Path("nn_1_challenge.keras")

# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [18]:
# Set the model's file path
file_path = Path("nn_1_challenge.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions[0:5]

In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=['predictions'], data=predictions)
predictions_df['predictions'] = round(predictions_df['predictions'], 0)
predictions_df

### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loan options, a variety of features must be collected to understand the student's financial needs, academic goals, and repayment capacity. Key demographic information such as age, citizenship status, state of residence, family size, and household income helps determine eligibility for different loan options, including federal and private loans, as well as need-based financial aid. Additionally, academic details like the student's school, field of study, enrollment status, and expected graduation year are crucial. These factors influence eligibility for specialized loan programs and repayment schedules, tailoring the system's recommendations based on the student's educational path.

Equally important is collecting financial information such as the student's expected family contribution (EFC), current income, outstanding loans, credit score, and savings, as this data helps assess their ability to repay loans without taking on excessive debt. Loan preferences, including the desired loan amount, repayment term, interest rate type, and flexibility in repayment, should also be considered to match the student's needs. Lastly, career goals and expected starting salary are essential to determine which loan options are most feasible based on the student’s future earning potential and eligibility for career-based loan forgiveness programs. Collecting all these data points ensures that the recommendation system can provide personalized, relevant, and affordable loan options.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

The recommendation system for student loan options would primarily use content-based filtering, which suggests loan options based on the characteristics of both the student and the loans themselves. Content-based filtering is ideal for this recommendation system because it focuses on matching the student’s unique profile to the features of different loan options. Unlike collaborative filtering, which relies on data from other users, content-based filtering uses the student's individual characteristics and financial needs to suggest the most suitable loans. By leveraging detailed attributes like demographics, academic background, and financial situation, the system can tailor recommendations without needing input from other students, ensuring that the suggestions are highly relevant and personalized.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

When building a recommendation system for student loans, there are several significant challenges that can impact its effectiveness and fairness. Two major concerns are data privacy and security, as well as ensuring fairness and reducing bias in loan recommendations. The first challenge involves the collection of sensitive personal information, such as financial details (income, credit score), academic records (school, field of study), and demographic data (age, citizenship status). Mishandling or breaching this data can lead to severe consequences like identity theft, financial fraud, or legal violations. If students do not trust the system to safeguard their personal data, they may be reluctant to share the information necessary for accurate loan recommendations, undermining the system's effectiveness. 

The second challenge is ensuring fairness and reducing bias in the recommendation system. If the model or the data used to train it contains biases, certain groups of students—such as those from lower-income or minority backgrounds—could be unfairly disadvantaged. For example, if credit score is heavily weighted, students with limited financial history may be excluded from loan opportunities they are otherwise eligible for. This could perpetuate financial inequality by offering higher-interest loans to disadvantaged students or by denying them access to affordable options altogether. 
